# Parameters estimation for a normal distribution (Python file)

For a population whose distribution is normal, we have to cases in bayesian statistics. One where we only determine the mean $\mu$, and the variance $\sigma^2$ is known; and one where both parameters are undetermined.

In [ ]:
import numpy as np
import scipy.special as sp


# First we get a random number between 0 and 100 for the mean (to estimate)
random_mean = 100*np.random.rand()
# Also one number between 0 and 10 for variance
random_variance = 10*np.random.rand()

# We need a function with a normal (multiple) distribution for the prior and posterior.
# The function will take an x values, a mean and a variance
def normal(X,mean,variance):
    return (2*np.pi*variance)**(-X/2) * np.exp(-(X-mean)**2)/(2*variance)

# We use a gamma distribution for the precision (1/variance) prior and posterior
def gamma(mu,alpha,beta):
    return (beta**alpha)/sp.gamma(alpha) * mu**(alpha-1) * np.exp(-beta*mu)

Now, in the following cells you can insert the parameters that you need for the priori and for the sample. You can allways replay this cell, which will show the graphs for the priori and posterior, the parameters, and info about the sample taken.cWe must ask the user if the variance will be or will be not known, so depending on the selected option, we use the proper update process.

Remember that the distribution for the mean $\mu$ is normal with mean $\mu_i$ (Where $i$ changes depending on prior or posterior), and std dev $s_i$. For the case where the variance is known, this two values are updated.

For the case where both the variance and the mean are unknown, $s_0^2$ takes the form $\sigma^2/\omega$ for the prior, where $\sigma^2$ is the variance to estimate and $\omega$ a guess to correlate the mean's distribution and the populatuion's distribution. Also, the variance $\sigma^2$ follows an inverse gamma distribution ($1/\sigma^2$) with parameters $k/2$ and $k \sigma_0^2 /2$, so that the expected value is $1/\sigma_0^2$, and the variance is $2/(k \sigma_0^4)$ (so $\sigma_0^2$ is the value we estimate for $\sigma^2$, and $k$ is the degree of confidence) 

In [ ]:
# We ask for the estimated mean
mu_prior = float(input("Enter the mean estimated for the population's distribution"))

# We ask if the process will be done with the variance known or unknown
variance_known = bool(input("The variance will be known? (0 or 1 for no and yes): "))
if variance_known == 1:
    # We ask for the variance (confidence) of the mean provided as a prior
    s_prior = float(input("Enter the variance for the mean prior distribution (confidence degree)"))
    # We reveal just the variance
    print(f'The variance for the population distribution is {random_variance}')
else:
    # We ask for the number of events to take the sample
    pass


# We ask for the parameters of the prior gamma distribution
a_prior = float(input("Enter the 'a' parameter of the prior Gamma distribution: "))
b_prior = float(input("Enter the 'b' parameter of the prior Gamma distribution: "))
#print'em
print(f"Using Gamma({a_prior}, {b_prior}) as prior distribution.")

# We if we will use poisson events or a fixed number of events to take the sample
use_poisson = bool(input("Do you want to use Poisson events? (0 or 1 for no and yes): "))
if use_poisson == 1:
    # We ask for the interval to take the sample
    t_interval = float(input("Enter the time interval to take the sample: "))
    # We get the number of events in that interval
    n_sample = np.random.poisson(random_number*t_interval)
    print(f"Number of events in interval {t_interval}: {n_sample}")
else:
    # We ask for the number of events to take the sample
    n_sample = int(input("Enter the number of events to take the sample: "))
    # We get the time interval for that number of events given the random number, using a gamma distribution
    # (we use 1/random_number as the scale parameter of the gamma)
    t_interval = np.random.gamma(n_sample,1/random_number)
    print(f"Time interval for {n_sample} events: {t_interval}")

# Now we can compute the posterior parameters
a_post = a_prior + n_sample
b_post = b_prior + t_interval
print(f"Posterior distribution is Gamma({a_post}, {b_post})")

# Then we graph the prior and posterior distributions
import matplotlib.pyplot as plt
mu_values = np.linspace(0, 100, 100)
prior = gamma(mu_values, a_prior, b_prior)
posterior = gamma(mu_values, a_post, b_post)
plt.plot(mu_values, prior, label=f"Prior: Gamma({a_prior}, {b_prior})", color='blue')
plt.plot(mu_values, posterior, label=f"Posterior: Gamma({a_post}, {b_post})", color='red')
plt.axvline(random_number, color='green', linestyle='--', label='True value')
plt.title('Prior and Posterior Distributions')
plt.xlabel("mu")
plt.ylabel("Density")
plt.legend()